<a href="https://colab.research.google.com/github/ssnirgudkar/Datasetpaper-final/blob/main/json_metrics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This code gets metrics from the images, how many images for each class, pixel size etc

In [1]:
import zipfile
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!mkdir "/content/jsonmetrics" 
 

In [3]:
# copy the select_test_images and masks in test and testannot respectively 
!cp /content/drive/MyDrive/DatasetMetricReport/revised_json/*.json /content/jsonmetrics 




In [4]:
import os 
APP_FOLDER = '/content/jsonmetrics'
totalFiles = 0
totalDir = 0

for base, dirs, files in os.walk(APP_FOLDER):
    print('Searching in : ',base)
    for directories in dirs:
        totalDir += 1
    for Files in files:
        totalFiles += 1
   

print('Total number of files',totalFiles)
print('Total Number of directories',totalDir)
print('Total:',(totalDir + totalFiles))

Searching in :  /content/jsonmetrics
Total number of files 55
Total Number of directories 0
Total: 55


In [7]:
import os
import sys
import argparse
import json

#annFile = "export_coco-panoptic_brunswick_IR-2019-10-08-14-49-48_v2.0.json"



def readJSONFile(fullFilePath):
    fileH = open(fullFilePath)
    data = json.load(fileH)
    #print(data["categories"]) # {'id': 0, 'name': 'sky', 'color': [0, 113, 188], 'isthing': 0}
    categoryMap = {}
    categoryMapName2Id = {}
    for eachCategory in data["categories"]:
        #print(eachCategory['id'], eachCategory['name'])
        categoryMap[eachCategory['id']] = eachCategory['name']
        categoryMapName2Id[eachCategory['name']] = eachCategory['id']

    images = data["images"]
    
    numImages2019 = 0
    numImages2020 = 0
    if("2019" in fullFilePath or "PilotIR" in fullFilePath or "CatchAll" in fullFilePath) :
      numImages2019 = len(images)
    else:
      numImages2020 = len(images)

    #print("fileName={0}, 2019={1}, 2020={2}\n".format(fullFilePath, numImages2019, numImages2020))

    #for eachAnnotation in data["annotations"]:
    #print(data["annotations"][0])
    bbox = data["annotations"][0]["segments_info"][0]['bbox']
    width = bbox[2] - bbox[0]
    height = bbox[3] - bbox[1]
    #print("width={0},height={1}\n".format(width, height))

    # fileRecord[0] = {'name':'foo.png', 'instance_record':[{}, {}, {}]}
    fileRecord = []
    index = 0
     


    for eachAnnotation in data["annotations"]:
        #print(eachAnnotation["file_name"])
        fileRecord.append({"file_name":eachAnnotation["file_name"]}) # one field struct
        fileRecord[index]["instance_record"] = [] # array of individual struct
        for eachSegInfo in eachAnnotation["segments_info"]:
            bbox = eachSegInfo["bbox"]
            width = bbox[2] 
            height = bbox[3] 
            categoryName = categoryMap[eachSegInfo["category_id"]]
            area = eachSegInfo["area"]
            fileRecord[index]["instance_record"].append({'bbox':bbox, 'width':width, 'height':height, 'categoryName':categoryName,'area':area})
            if area <10 and eachSegInfo["category_id"] == 2:
              print ("file where bridge is <10 is {0} and dataset is {1}", fullFilePath, eachAnnotation["file_name"])
            if area <3 and eachSegInfo["category_id"] == 3:
              print ("file where obstacle is <3 is {0} and dataset is {1}", fullFilePath, eachAnnotation["file_name"])
            if area <2 and eachSegInfo["category_id"] == 4:
              print ("file where living ob is <2 is {0} and dataset is {1}", fullFilePath, eachAnnotation["file_name"])
                     


        index = index+1
    
    
    #for eachFileRecord in fileRecord:
    #    print(eachFileRecord)
    return (fileRecord, numImages2019, numImages2020)

def metricfunction(args):
    fileRecords = []
    total2019Images = 0
    total2020Images = 0
    if ("json_file" in args != None):
        fileRecords.append(readJSONFile(args["json_file"]))
    elif(args["json_dir"] != None):
        gen = (eachFile for eachFile in os.listdir(args["json_dir"]) if eachFile.endswith(".json"))
        
        for eachFile in gen:
            fullFilePath = os.path.join(args["json_dir"], eachFile)
            (fileRecord, numImages2019, numImages2020) = readJSONFile(fullFilePath)
            fileRecords.append(fileRecord)
            total2019Images = total2019Images + numImages2019
            total2020Images = total2020Images + numImages2020


    LessThan10x10 = []
    Between10x10And20x20 = []
    Between20x20And30x30 = []
    Between30x30And40x40 = []
    Between40x40And50x50 = []
    Between60x60And70x70 = []
    Between80x80And90x90 = []
    Between90x90And100x100 = []
    Between100x100And150x150 = []
    Between150x150And200x200 = []
    Between200x200And250x250 = []
    Between250x250And300x300 = []
    Between300x300And350x350 = []
    Between350x350And400x400 = []

    # Small : Bounding box in X and in Y < 100
    # Medium : Bounding box in X and in Y between 100 and 400
    # Large : Bounding box in X and in Y greater than 400
    numSmall = 0
    numMedium = 0
    numLarge = 0

    #Initialize counters 
    bridge_tiny = 0
    ob_tiny = 0 
    living_ob_tiny = 0 
    sky_tiny = 0 
    water_tiny = 0 
    background_tiny = 0 
    self_tiny = 0 


    bridge_small = 0 
    ob_small = 0 
    living_ob_small = 0
    sky_small = 0 
    water_small = 0 
    background_small = 0
    self_small = 0 

    bridge_medium = 0
    ob_medium =0 
    living_ob_medium = 0
    sky_medium = 0 
    water_medium = 0 
    background_medium = 0 
    self_medium = 0 


    bridge_large = 0
    ob_large = 0 
    living_ob_large = 0 
    sky_large = 0 
    water_large = 0 
    background_large = 0 
    self_large = 0 

    bridge_xlarge = 0
    ob_xlarge = 0  
    living_ob_xlarge = 0 
    sky_xlarge = 0
    water_xlarge = 0 
    background_xlarge = 0 
    self_xlarge = 0 

    bridge_xxlarge = 0
    ob_xxlarge = 0 
    living_ob_xxlarge = 0
    sky_xxlarge = 0 
    water_xxlarge = 0 
    background_xxlarge = 0 
    self_xxlarge = 0 

    # Smallest and Largest in each category.
    minArea = [10000000, 10000000, 10000000, 10000000, 10000000, 10000000, 10000000]
    maxArea = [0, 0, 0, 0, 0, 0, 0]
    for eachFileRecord in fileRecords:
        for instanceRecords in eachFileRecord:
            for eachRecord in instanceRecords["instance_record"]:
                #print(eachRecord["width"], eachRecord["height"], eachRecord["area"])
                if (eachRecord["area"] < 1000):
                    numSmall = numSmall + 1
                elif ( eachRecord["area"] <  10000):
                    numMedium = numMedium + 1
                else:
                    numLarge = numLarge + 1
                
                if ("sky" == eachRecord["categoryName"]):
                    if (eachRecord["area"] < minArea[0]):
                        minArea[0] = eachRecord["area"]
                    elif (eachRecord["area"] > maxArea[0]):
                        maxArea[0] = eachRecord["area"]
                elif ("water" == eachRecord["categoryName"]):
                    if (eachRecord["area"] < minArea[1]):
                        minArea[1] = eachRecord["area"]
                    elif (eachRecord["area"] > maxArea[1]):
                        maxArea[1] = eachRecord["area"]
                elif ("bridge" == eachRecord["categoryName"]):
                    if (eachRecord["area"] < minArea[2] and eachRecord["area"] != 1) :
                        minArea[2] = eachRecord["area"]
                    elif (eachRecord["area"] > maxArea[2]):
                        maxArea[2] = eachRecord["area"]
                elif ("obstacle" == eachRecord["categoryName"]):
                    if (eachRecord["area"] < minArea[3] and eachRecord["area"] != 1):
                        minArea[3] = eachRecord["area"]
                    elif (eachRecord["area"] > maxArea[3]):
                        maxArea[3] = eachRecord["area"]
                elif ("living obstacle" == eachRecord["categoryName"]):
                    if (eachRecord["area"] < minArea[4] and eachRecord["area"] != 1):
                        minArea[4] = eachRecord["area"]
                    elif (eachRecord["area"] > maxArea[4]):
                        maxArea[4] = eachRecord["area"]
                elif ("background" == eachRecord["categoryName"]):
                    if (eachRecord["area"] < minArea[5]):
                        minArea[5] = eachRecord["area"]
                    elif (eachRecord["area"] > maxArea[5]):
                        maxArea[5] = eachRecord["area"]
                elif ("self" == eachRecord["categoryName"]):
                    if (eachRecord["area"] < minArea[6]):
                        minArea[6] = eachRecord["area"]
                    elif (eachRecord["area"] > maxArea[6]):
                        maxArea[6] = eachRecord["area"]

                # count number of images for ob, living ob and bridge  
                if eachRecord["categoryName"] == 'bridge' :
                  if 0 <= eachRecord["area"] <100: 
                      bridge_tiny = bridge_tiny+1 
                  if 100 <= eachRecord["area"] < 300: 
                      bridge_small = bridge_small+1 
                  if 300 <= eachRecord["area"] < 1000: 
                      bridge_medium = bridge_medium+1 
                  if 1000 <= eachRecord["area"] < 30000: 
                      bridge_large = bridge_large+1 
                  if 30000 <= eachRecord["area"] < 100000: 
                      bridge_xlarge = bridge_xlarge+1 
                  if eachRecord["area"] >= 100000: 
                      bridge_xxlarge = bridge_xxlarge+1

                if eachRecord["categoryName"] == "obstacle" :
                  if 0 <= eachRecord["area"] < 100: 
                      ob_tiny = ob_tiny+1 
                  if 100 <= eachRecord["area"] < 300: 
                      ob_small = ob_small+1 
                  if 300 <= eachRecord["area"] < 1000: 
                     ob_medium = ob_medium+1 
                  if 1000 <= eachRecord["area"] < 30000: 
                     ob_large = ob_large+1 
                  if 30000 <= eachRecord["area"] < 100000: 
                     ob_xlarge = ob_xlarge+1 
                  if eachRecord["area"] >100000: 
                     ob_xxlarge = ob_xxlarge+1

                if eachRecord["categoryName"] == "living obstacle" :
                  if 0 <= eachRecord["area"] < 100: 
                     living_ob_tiny = living_ob_tiny+1 
                  if 100 <= eachRecord["area"] < 300: 
                      living_ob_small = living_ob_small+1 
                  if 300 <= eachRecord["area"] < 1000: 
                      living_ob_medium = living_ob_medium+1 
                  if 1000 <= eachRecord["area"] < 30000: 
                      living_ob_large = living_ob_large+1 
                  if 30000 <= eachRecord["area"] < 100000: 
                     living_ob_xlarge = living_ob_xlarge+1 
                  if eachRecord["area"] > 100000: 
                     living_ob_xxlarge = living_ob_xxlarge+1

                if eachRecord["categoryName"] == "sky" :
                  if 0 <= eachRecord["area"] < 100: 
                     sky_tiny = sky_tiny+1 
                  if 100 <= eachRecord["area"] < 300: 
                      sky_small = sky_small+1 
                  if 300 <= eachRecord["area"] < 1000: 
                      sky_medium = sky_medium+1 
                  if 1000 <= eachRecord["area"] < 30000: 
                      sky_large = sky_large+1 
                  if 30000 <= eachRecord["area"] < 100000: 
                     sky_xlarge = sky_xlarge+1 
                  if eachRecord["area"] > 100000: 
                     sky_xxlarge = sky_xxlarge+1

                if eachRecord["categoryName"] == "water" :
                  if 0 <= eachRecord["area"] < 100: 
                     water_tiny = water_tiny+1 
                  if 100 <= eachRecord["area"] < 300: 
                      water_small = water_small+1 
                  if 300 <= eachRecord["area"] < 1000: 
                      water_medium = water_medium+1 
                  if 1000 <= eachRecord["area"] < 30000: 
                      water_large = water_large+1 
                  if 30000 <= eachRecord["area"] < 100000: 
                     water_xlarge = water_xlarge+1 
                  if eachRecord["area"] > 100000: 
                     water_xxlarge = water_xxlarge+1
            
                if eachRecord["categoryName"] == "background" :
                  if 0 <= eachRecord["area"] < 100: 
                     background_tiny = background_tiny+1 
                  if 100 <= eachRecord["area"] < 300: 
                      background_small = background_small+1 
                  if 300 <= eachRecord["area"] < 1000: 
                      background_medium = background_medium+1 
                  if 1000 <= eachRecord["area"] < 30000: 
                      background_large = background_large+1 
                  if 30000 <= eachRecord["area"] < 100000: 
                     background_xlarge = background_xlarge+1 
                  if eachRecord["area"] > 100000: 
                     background_xxlarge = background_xxlarge+1

                if eachRecord["categoryName"] == "self" :
                  if 0 <= eachRecord["area"] < 100: 
                     self_tiny = self_tiny+1 
                  if 100 <= eachRecord["area"] < 300: 
                      self_small = self_small+1 
                  if 300 <= eachRecord["area"] < 1000: 
                      self_medium = self_medium+1 
                  if 1000 <= eachRecord["area"] < 30000: 
                      self_large = self_large+1 
                  if 30000 <= eachRecord["area"] < 100000: 
                     self_xlarge = self_xlarge+1 
                  if eachRecord["area"] > 100000: 
                     self_xxlarge = self_xxlarge+1
                                        

    print("Small items={0}, Medium items={1}, Large items={2}".format(numSmall, numMedium, numLarge))
    print("Sky Min area={0}, Max area={1}".format(minArea[0], maxArea[0]))
    print("Water Min area={0}, Max area={1}".format(minArea[1], maxArea[1]))
    print("Bridge Min area={0}, Max area={1}".format(minArea[2], maxArea[2]))
    print("Obstacle Min area={0}, Max area={1}".format(minArea[3], maxArea[3]))
    print("Living obstacle Min area={0}, Max area={1}".format(minArea[4], maxArea[4]))
    print("Background Min area={0}, Max area={1}".format(minArea[5], maxArea[5]))
    print("Self Min area={0}, Max area={1}".format(minArea[6], maxArea[6]))

    # Now print the cummulative numbers for bridge (pixel count per instance)
    print ("bridge_tiny:", bridge_tiny)
    print ("bridge_small:", bridge_small )
    print ("bridge_medium:", bridge_medium )
    print ("bridge_large:", bridge_large )
    print ("bridge_xlarge:", bridge_xlarge)
    print ("bridge_xxlarge:", bridge_xxlarge)

    # Now print the cummulative numbers for obstacles pixel count per instance
    print ("ob_tiny:", ob_tiny)
    print ("ob_small:", ob_small )
    print ("ob_medium:", ob_medium )
    print ("ob_large:", ob_large )
    print ("ob_xlarge:", ob_xlarge)
    print ("ob_xxlarge:", ob_xxlarge)


    # Now print the cummulative numbers for living obstacles pixel count per instance
    print ("living_ob_tiny:", living_ob_tiny)
    print ("living_ob_small:", living_ob_small )
    print ("living_ob_medium:", living_ob_medium )
    print ("living_ob_large:", living_ob_large )
    print ("living_ob_xlarge:", living_ob_xlarge)
    print ("living_ob_xxlarge:", living_ob_xxlarge)

    # Now print the cummulative numbers for sky pixel count per instance
    print ("sky_tiny:", sky_tiny)
    print ("sky_small:", sky_small )
    print ("sky_medium:", sky_medium )
    print ("sky_large:", sky_large )
    print ("sky_xlarge:", sky_xlarge)
    print ("sky_xxlarge:", sky_xxlarge)

    # Now print the cummulative numbers for water pixel count per instance  
    print ("water_tiny:", water_tiny)
    print ("water_small:", water_small )
    print ("water_medium:", water_medium )
    print ("water_large:", water_large )
    print ("water_xlarge:", water_xlarge)
    print ("water_xxlarge:", water_xxlarge)

    # Now print the cummulative numbers for background pixel count per instance 
    print ("background_tiny:", background_tiny)
    print ("background_small:", background_small )
    print ("background_medium:", background_medium )
    print ("background_large:", background_large )
    print ("background_xlarge:", background_xlarge)
    print ("background_xxlarge:", background_xxlarge)

    # Now print the cummulative numbers for self  pixel count per instance
    print ("self_tiny:", self_tiny)
    print ("self_small:", self_small )
    print ("self_medium:", self_medium )
    print ("self_large:", self_large )
    print ("self_xlarge:", self_xlarge)
    print ("self_xxlarge:", self_xxlarge)
                
    print("Total 2019 images = {0}\n".format(total2019Images))
    print("Total 2020 images = {0}\n".format(total2020Images))
    print("Total images = {0}\n".format(total2019Images+total2020Images))


In [8]:
args = {}
args["json_dir"] = "/content/jsonmetrics"
metricfunction(args)

file where obstacle is <3 is {0} and dataset is {1} /content/jsonmetrics/export_coco-panoptic_ssnirgudkar_IR-2020-10-22-17-33-25-0_v2.0.json 1603389110.873336_label_ground-truth_coco-panoptic.png
file where obstacle is <3 is {0} and dataset is {1} /content/jsonmetrics/export_coco-panoptic_brunswick_IR-RT-2021-10-28-18-38-20_0_v2.0.json 1635461369.002067_label_ground-truth_coco-panoptic.png
file where obstacle is <3 is {0} and dataset is {1} /content/jsonmetrics/export_coco-panoptic_brunswick_IR-catch-all_v2.0.json 1602782510.007807_label_ground-truth_coco-panoptic.png
file where obstacle is <3 is {0} and dataset is {1} /content/jsonmetrics/export_coco-panoptic_ssnirgudkar_PilotIR_v2.0.json 1602782542.023886_label_segmentation_coco-panoptic.png
file where obstacle is <3 is {0} and dataset is {1} /content/jsonmetrics/export_coco-panoptic_ssnirgudkar_PilotIR_v2.0.json 1602782542.023886_label_segmentation_coco-panoptic.png
file where obstacle is <3 is {0} and dataset is {1} /content/jsonme